In [1]:
import ee
import geopandas as gpd
import geemap
import xarray as xr
import wxee

In [2]:
ee.Authenticate()
ee.Initialize(project='ee-nunezrimedio-tesina',opt_url='https://earthengine-highvolume.googleapis.com')

In [3]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

def get_province_geometry(province_name):
    provinces = ee.FeatureCollection("FAO/GAUL/2015/level1")
    return provinces.filter(ee.Filter.And(
        ee.Filter.eq('ADM0_NAME', 'Argentina'),
        ee.Filter.eq('ADM1_NAME', province_name)
    )).first().geometry()

# Get geometries for Santiago del Estero and Chaco
santiago_geometry = get_province_geometry('Santiago Del Estero')
chaco_geometry = get_province_geometry('Chaco')

# Find an area on the border
border_area = santiago_geometry.intersection(chaco_geometry).centroid().buffer(35000).bounds()

parana_delta = ee.Geometry.Polygon(
    [[[-59.5, -33.5],
      [-59.5, -34.0],
      [-58.8, -34.0],
      [-58.8, -33.5]]])

# Create a map centered on Argentina
Map = geemap.Map(center=[-27, -62], zoom=6)

# Add layers to the map
Map.addLayer(border_area, {'color': 'FF0000', 'fillColor': 'FF000088'}, 'Border Area')
Map.addLayer(parana_delta, {'color': '0000FF', 'fillColor': '0000FF88'}, 'Paraná Delta')

# Display the map
Map

Map(center=[-27, -62], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

## Descarga Basica: Moda de la banda label

In [8]:
aoi = parana_delta
dw_filtered = wxee.TimeSeries("GOOGLE/DYNAMICWORLD/V1").filterDate("2015-01-01", "2016-01-01").filterBounds(aoi).select("label")

def clip_image(image):
    return image.clip(aoi)

dw_filtered = dw_filtered.map(clip_image)

monthly_ts = dw_filtered.aggregate_time("year", ee.Reducer.mode())

ds = monthly_ts.wx.to_xarray(region=ee.Feature(aoi).geometry(), scale=30)
ds = ds.fillna(-1)
ds = ds.clip(min=-128,max=127)
ds["label"] = ds["label"].astype("int8")
ds["x"] = ds["x"].astype("float32")
ds["y"] = ds["y"].astype("float32")
ds["time"] = ds["time"].dt.strftime("%Y-%m")
ds.to_netcdf(rf"D:\Factor-Data\Tesina\Tesina\Validation\2015_parana_delta_mode.nc")

Requesting data:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
aoi = border_area
dw_filtered = wxee.TimeSeries("GOOGLE/DYNAMICWORLD/V1").filterDate("2015-01-01", "2016-01-01").filterBounds(aoi).select("label")

def clip_image(image):
    return image.clip(aoi)

dw_filtered = dw_filtered.map(clip_image)

monthly_ts = dw_filtered.aggregate_time("year", ee.Reducer.mode())

ds = monthly_ts.wx.to_xarray(region=ee.Feature(aoi).geometry(), scale=30)
ds = ds.fillna(-1)
ds = ds.clip(min=-128,max=127)
ds["label"] = ds["label"].astype("int8")
ds["x"] = ds["x"].astype("float32")
ds["y"] = ds["y"].astype("float32")
ds["time"] = ds["time"].dt.strftime("%Y-%m")
ds.to_netcdf(rf"D:\Factor-Data\Tesina\Tesina\Validation\2015_border_area_mode.nc")

Requesting data:   0%|          | 0/1 [00:00<?, ?it/s]

## Agregación Espacial de la banda label con moda

In [10]:
aoi = parana_delta

# Load the Dynamic World dataset and filter by AOI and date range
dw_filtered = wxee.TimeSeries("GOOGLE/DYNAMICWORLD/V1") \
                .filterDate("2015-01-01", "2016-01-01") \
                .filterBounds(aoi) \
                .select("label")

# Clip images to the area of interest
def clip_image(image):
    return image.clip(aoi)

# Apply the clipping function to each image in the collection
dw_filtered = dw_filtered.map(clip_image)

# Aggregate temporally to get the mode for each year
yearly_ts = dw_filtered.aggregate_time("year", ee.Reducer.mode())

# Function to spatially pool the data using a 30m resolution
def spatial_pool(image):
    # Set the default projection for the image
    image = image.setDefaultProjection('EPSG:4326', None, 30)  # Set the projection with a scale of 30m

    return image.reduceResolution(
        reducer=ee.Reducer.mode(),
        maxPixels=65535,  # Ensure this is less than 65536
        bestEffort=True
    ).reproject(crs='EPSG:4326', scale=30)

# Apply spatial pooling to each image in the yearly collection
pooled_collection = yearly_ts.map(spatial_pool)

# Download the pooled image collection directly as xarray using wxee
# Convert the pooled collection to xarray
# Ensure to specify the region and scale
ds = pooled_collection.wx.to_xarray(region=aoi, scale=30)

# Fill NaN values and clip to appropriate ranges
ds = ds.fillna(-1)
ds = ds.clip(min=-128, max=127)
ds["label"] = ds["label"].astype("int8")
ds["x"] = ds["x"].astype("float32")
ds["y"] = ds["y"].astype("float32")
ds["time"] = ds["time"].dt.strftime("%Y-%m")

# Save the pooled dataset back to NetCDF
ds.to_netcdf(r"D:\Factor-Data\Tesina\Tesina\Validation\2015_parana_delta_mode_pooled_30m.nc")


Requesting data:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
aoi = border_area

# Load the Dynamic World dataset and filter by AOI and date range
dw_filtered = wxee.TimeSeries("GOOGLE/DYNAMICWORLD/V1") \
                .filterDate("2015-01-01", "2016-01-01") \
                .filterBounds(aoi) \
                .select("label")

# Clip images to the area of interest
def clip_image(image):
    return image.clip(aoi)

# Apply the clipping function to each image in the collection
dw_filtered = dw_filtered.map(clip_image)

# Aggregate temporally to get the mode for each year
yearly_ts = dw_filtered.aggregate_time("year", ee.Reducer.mode())

# Function to spatially pool the data using a 30m resolution
def spatial_pool(image):
    # Set the default projection for the image
    image = image.setDefaultProjection('EPSG:4326', None, 30)  # Set the projection with a scale of 30m

    return image.reduceResolution(
        reducer=ee.Reducer.mode(),
        maxPixels=65535,  # Ensure this is less than 65536
        bestEffort=True
    ).reproject(crs='EPSG:4326', scale=30)

# Apply spatial pooling to each image in the yearly collection
pooled_collection = yearly_ts.map(spatial_pool)

# Download the pooled image collection directly as xarray using wxee
# Convert the pooled collection to xarray
# Ensure to specify the region and scale
ds = pooled_collection.wx.to_xarray(region=aoi, scale=30)

# Fill NaN values and clip to appropriate ranges
ds = ds.fillna(-1)
ds = ds.clip(min=-128, max=127)
ds["label"] = ds["label"].astype("int8")
ds["x"] = ds["x"].astype("float32")
ds["y"] = ds["y"].astype("float32")
ds["time"] = ds["time"].dt.strftime("%Y-%m")

# Save the pooled dataset back to NetCDF
ds.to_netcdf(r"D:\Factor-Data\Tesina\Tesina\Validation\2015_border_area_mode_pooled_30m.nc")

Requesting data:   0%|          | 0/1 [00:00<?, ?it/s]

## Agregación Espacial de todas las bandas con media


In [12]:
aoi = parana_delta

CLASS_NAMES = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built', 'bare', 'snow_and_ice']
CLASS_NUMBER = ee.List.sequence(0, 8)

# Load the Dynamic World dataset and filter by AOI and date range
dw_filtered = wxee.TimeSeries("GOOGLE/DYNAMICWORLD/V1") \
    .filterDate("2015-01-01", "2016-01-01") \
    .filterBounds(aoi)

# Clip images to the area of interest
def clip_image(image):
    return image.clip(aoi)

# Apply the clipping function to each image in the collection
dw_filtered = dw_filtered.map(clip_image)

# Aggregate temporally to get the mode for each year
yearly_ts = dw_filtered.aggregate_time("year", ee.Reducer.mean())

# Function to spatially pool the data using a 30m resolution
def spatial_pool(image):
    # Set the default projection for the image
    image = image.setDefaultProjection('EPSG:4326', None, 30)  # Set the projection with a scale of 30m

    return image.reduceResolution(
        reducer=ee.Reducer.mean(),
        maxPixels=65535,  # Ensure this is less than 65536
        bestEffort=True
    ).reproject(crs='EPSG:4326', scale=30)

# Apply spatial pooling to each image in the yearly collection
pooled_collection = yearly_ts.map(spatial_pool)


yearly_dominant_class = pooled_collection.map(
    lambda image:
    image.select(CLASS_NAMES)
    .reduce(ee.Reducer.max())
    .eq(image.select(CLASS_NAMES))
    .multiply(ee.Image.constant(CLASS_NUMBER))
    .reduce(ee.Reducer.sum())
    .rename('label')
    .uint8()
    .copyProperties(image, ['system:time_start'])
)


# Download the pooled image collection directly as xarray using wxee
# Convert the pooled collection to xarray
# Ensure to specify the region and scale
ds = yearly_dominant_class.wx.to_xarray(region=aoi, scale=30)

# Fill NaN values and clip to appropriate ranges
ds = ds.fillna(-1)
ds = ds.clip(min=-128, max=127)
ds["label"] = ds["label"].astype("int8")
ds["x"] = ds["x"].astype("float32")
ds["y"] = ds["y"].astype("float32")
ds["time"] = ds["time"].dt.strftime("%Y-%m")

# Save the pooled dataset back to NetCDF
ds.to_netcdf(r"D:\Factor-Data\Tesina\Tesina\Validation\2015_parana_delta_mean_pooled_30m.nc")

Requesting data:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
aoi = border_area

CLASS_NAMES = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built', 'bare', 'snow_and_ice']
CLASS_NUMBER = ee.List.sequence(0, 8)

# Load the Dynamic World dataset and filter by AOI and date range
dw_filtered = wxee.TimeSeries("GOOGLE/DYNAMICWORLD/V1") \
    .filterDate("2015-01-01", "2016-01-01") \
    .filterBounds(aoi)

# Clip images to the area of interest
def clip_image(image):
    return image.clip(aoi)

# Apply the clipping function to each image in the collection
dw_filtered = dw_filtered.map(clip_image)

# Aggregate temporally to get the mode for each year
yearly_ts = dw_filtered.aggregate_time("year", ee.Reducer.mean())

# Function to spatially pool the data using a 30m resolution
def spatial_pool(image):
    # Set the default projection for the image
    image = image.setDefaultProjection('EPSG:4326', None, 30)  # Set the projection with a scale of 30m

    return image.reduceResolution(
        reducer=ee.Reducer.mean(),
        maxPixels=65535,  # Ensure this is less than 65536
        bestEffort=True
    ).reproject(crs='EPSG:4326', scale=30)

# Apply spatial pooling to each image in the yearly collection
pooled_collection = yearly_ts.map(spatial_pool)


yearly_dominant_class = pooled_collection.map(
    lambda image:
    image.select(CLASS_NAMES)
    .reduce(ee.Reducer.max())
    .eq(image.select(CLASS_NAMES))
    .multiply(ee.Image.constant(CLASS_NUMBER))
    .reduce(ee.Reducer.sum())
    .rename('label')
    .uint8()
    .copyProperties(image, ['system:time_start'])
)


# Download the pooled image collection directly as xarray using wxee
# Convert the pooled collection to xarray
# Ensure to specify the region and scale
ds = yearly_dominant_class.wx.to_xarray(region=aoi, scale=30)

# Fill NaN values and clip to appropriate ranges
ds = ds.fillna(-1)
ds = ds.clip(min=-128, max=127)
ds["label"] = ds["label"].astype("int8")
ds["x"] = ds["x"].astype("float32")
ds["y"] = ds["y"].astype("float32")
ds["time"] = ds["time"].dt.strftime("%Y-%m")

# Save the pooled dataset back to NetCDF
ds.to_netcdf(r"D:\Factor-Data\Tesina\Tesina\Validation\2015_border_area_mean_pooled_30m.nc")

Requesting data:   0%|          | 0/1 [00:00<?, ?it/s]

## Agregación Temporal

## Descarga V2: Media de todas las bandas y computo "label" de nuevo

In [13]:
import os
import ee
import wxee

# Initialize the AOI
aoi = border_area

# Class names and corresponding numbers
CLASS_NAMES = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built', 'bare', 'snow_and_ice']
CLASS_NUMBER = ee.List.sequence(0, 8)

# Load the Dynamic World dataset and filter by date and AOI
dw_filtered = wxee.TimeSeries("GOOGLE/DYNAMICWORLD/V1").filterDate("2014", "2026").filterBounds(aoi)

# Clip images to the AOI and retain the system:time_start property
dw_filtered = dw_filtered.map(lambda image: image.clip(aoi).copyProperties(image, ['system:time_start']))

# Aggregate the images by year and compute the mean for each year
yearly_ts = dw_filtered.aggregate_time(frequency="year", reducer=ee.Reducer.mean())

# Ensure time property exists for each image
yearly_ts = yearly_ts.map(lambda image: 
                          image.set('system:time_start', image.get('system:time_start') 
                                    or ee.Date.fromYMD(ee.Number(image.get('year')), 1, 1).millis() 
                                    or ee.Date('2000-01-01').millis()))

# Compute the dominant class for each image
yearly_dominant_class = yearly_ts.map(
    lambda image: 
    image.select(CLASS_NAMES)
         .reduce(ee.Reducer.max())
         .eq(image.select(CLASS_NAMES))
         .multiply(ee.Image.constant(CLASS_NUMBER))
         .reduce(ee.Reducer.sum())
         .rename('dominant_class')
         .uint8()
         .copyProperties(image, ['system:time_start'])
)
yearly_dominant_class.getInfo()


{'type': 'ImageCollection',
 'bands': [],
 'id': 'GOOGLE/DYNAMICWORLD/V1',
 'features': [{'type': 'Image',
   'bands': [{'id': 'dominant_class',
     'data_type': {'type': 'PixelType',
      'precision': 'int',
      'min': 0,
      'max': 255},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'properties': {'system:time_start': 1439993888000,
    'system:index': '20150819T141046_20150819T141808_T20JNR'}},
  {'type': 'Image',
   'bands': [{'id': 'dominant_class',
     'data_type': {'type': 'PixelType',
      'precision': 'int',
      'min': 0,
      'max': 255},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'properties': {'system:time_start': 1474553441460,
    'system:index': '20160922T141042_20160922T141041_T20JNR'}},
  {'type': 'Image',
   'bands': [{'id': 'dominant_class',
     'data_type': {'type': 'PixelType',
      'precision': 'int',
      'min': 0,
      'max': 255},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1,

In [12]:
aoi = parana_delta
dw_filtered = wxee.TimeSeries("GOOGLE/DYNAMICWORLD/V1").filterDate("2014", "2026").filterBounds(aoi).select("label")

def clip_image(image):
    return image.clip(aoi)

dw_filtered = dw_filtered.map(clip_image)

dw_filtered.aggregate_time("year", ee.Reducer.mode()).getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'id': 'GOOGLE/DYNAMICWORLD/V1',
 'features': [{'type': 'Image',
   'bands': [{'id': 'label',
     'data_type': {'type': 'PixelType',
      'precision': 'int',
      'min': 0,
      'max': 255},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'version': 1729347261159394,
   'id': 'GOOGLE/DYNAMICWORLD/V1/20150803T140146_20150803T140149_T21HTC',
   'properties': {'system:asset_size': 2416855344,
    'system:index': '20150803T140146_20150803T140149_T21HTC',
    'system:time_start': 1438610509000,
    'dynamicworld_algorithm_version': '3.5',
    'system:footprint': {'type': 'Polygon',
     'coordinates': [[[-59.06189439208878, -34.00046418653865],
       [-59.0598673270729, -33.91687670926022],
       [-59.04995550747806, -33.500451820566816],
       [-59.5, -33.49999999999999],
       [-59.5, -34],
       [-59.06189439208878, -34.00046418653865]]]},
    'qa_algorithm_version': '1'}},
  {'type': 'Image',
   'bands': [{'id': 

In [ ]:
"""
# Convert the result to an xarray dataset
ds = yearly_dominant_class.wx.to_xarray(region=aoi, scale=30)

# Retain only the 'dominant_class' band and ensure appropriate data formatting
ds = ds[['dominant_class']]
ds = ds.fillna(-1)
ds = ds.clip(min=0, max=8)
ds["dominant_class"] = ds["dominant_class"].astype("uint8")
ds["time"] = ds["time"].dt.strftime("%Y")

# Define the output directory and ensure it exists
output_dir = r"D:\Factor-Data\Tesina\Tesina\Validation"
os.makedirs(output_dir, exist_ok=True)

# Compression settings
encoding = {"dominant_class": {"zlib": True, "complevel": 5}}

# Save the dataset as a NetCDF file
output_file = os.path.join(output_dir, "border_area_dominant_class.nc")
ds.to_netcdf(output_file, encoding=encoding)

print(f"Completed processing border area. Output saved to {output_file}")
"""

In [ ]:
# Assuming border_area is defined earlier in your code
aoi = parana_delta

# Define class names
CLASS_NAMES = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built', 'bare', 'snow_and_ice']

# Filter and prepare the Dynamic World dataset
dw_filtered = wxee.TimeSeries("GOOGLE/DYNAMICWORLD/V1").filterDate("2014", "2026").filterBounds(aoi)

def clip_image(image):
    return image.clip(aoi)

dw_filtered = dw_filtered.map(clip_image)

# Aggregate time series to yearly mean
yearly_ts = dw_filtered.aggregate_time("year", ee.Reducer.mean())

# Convert to xarray
ds = yearly_ts.wx.to_xarray(region=ee.Feature(aoi).geometry(), scale=30)

# Ensure data types for coordinate variables
ds["x"] = ds["x"].astype("float32")
ds["y"] = ds["y"].astype("float32")
ds["time"] = ds["time"].dt.strftime("%Y")

# Compute dominant class
def compute_dominant_class(ds):
    class_values = ds[CLASS_NAMES].to_array().values
    dominant_class = np.argmax(class_values, axis=0)
    return xr.DataArray(dominant_class, coords=ds.coords, dims=ds.dims)

ds["label"] = compute_dominant_class(ds)
ds["label"] = ds["label"].astype("int8")

# Now fill NaN values and ensure data types
ds = ds.fillna(-1)
for band in CLASS_NAMES:
    ds[band] = ds[band].clip(min=0, max=1)
    ds[band] = ds[band].astype("float32")

ds["label"] = ds["label"].clip(min=-128, max=127)  # Ensure int8 range

# Save to NetCDF
ds.to_netcdf(r"D:\Factor-Data\Tesina\Tesina\Validation\parana_delta_mean_dominant.nc")

## Análisis